### Web Search Agent 1.

## Simple Web Search Agent using duckduckgo API

This agent:
- uses the duckduckgo API (DuckDuckGo Instant Answer API)
- Validates queries using pydantic
- Has error handling with some fallbacks

In [3]:
import requests
from typing import List, Optional
from pydantic import BaseModel, ValidationError
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")

Note: this API doesn't require any API key, just a module called `ddgs` to be used.

### Defining the Pydantic model for the search query
This schema strictly validates and structures the web search results, applications down the line can use this to get a trusted format and catch errors

In [4]:
class SearchResult(BaseModel):
    title: str
    url: str
    snippet: Optional[str]

### Web Search Function (DuckDuckGo API)

- Queries the API
- If the result set is empty or an error occurs, returns a single informative result.
- Exception handling is used for API errors, connection failures, and schema problems.


In [21]:
from ddgs import DDGS

def web_search_duckduckgo(query: str, max_results: int = 5) -> List[SearchResult]:
    results = []
    try:
        with DDGS() as ddgs:
            for r in ddgs.text(query, max_results=max_results):
                snippet = r.get("body", "")
                if len(snippet) > 200:  # truncate to 200 chars
                    snippet = snippet[:200] + "..."
                results.append(SearchResult(
                    title=r.get("title", "No Title"),
                    url=r.get("href", ""),
                    snippet=snippet
                ))
        if not results:
            results.append(SearchResult(
                title="No Results Found",
                url="",
                snippet="No relevant results were found for your query."
            ))
        return results
    except Exception as e:
        return [
            SearchResult(
                title="Web Search Error",
                url="",
                snippet=f"An error occurred: {str(e)}"
            )
        ]


The current fallback in the `web_search` function is just to say no results found when theres no results and to give the error messsage when an error has occured. Some ideas:
- Making a different function `web_search_with_llm` that uses an LLM call as a fallback in case there is no search results found. 

### Testing the Agent


In [22]:
query = "What is a group of crows called?"
results = web_search(query)
for idx, res in enumerate(results, 1):
    print(f"{idx}. {res.title}\n{res.url}\n{res.snippet}\n")


1. Why Is A Group Of Crows Called A Murder? | Bird Spot
https://www.birdspot.co.uk/bird-brain/why-is-a-group-of-crows-called-a-murder
Crows are also known by a variety of other collective nouns, including a horde, a hover, a mob, a parcel, a parliament, and a storytelling. The term parliament is also commonly used to describe a group of owls.

2. What is a Group of Crows Called ?
https://s3.amazonaws.com/external_clips/4535917/What_is_a_Group_of_Crows_Called_.pdf?1677863847
A group of crows is called a murder. There are many different accounts for where this term came from, most of those being based on superstitions and folk tales. A group of crows is called a murder due to collective nouns being popular during medieval times.

3. What is a Group of Crows Called & Why? [Murder...] | World Birds
https://worldbirds.com/what-is-a-group-of-crows-called/
Murder, mob, or horde – what is a group of crows called ? English can be a funny language, and with over 900 collective nouns, it’s bound 

In [23]:
query = "How many seasons did it take for Ted to introduce the mother in how I met your mother?"
results = web_search(query)
for idx, res in enumerate(results, 1):
    print(f"{idx}. {res.title}\n{res.url}\n{res.snippet}\n")


1. MANY Definition & Meaning - Merriam-Webster
https://www.merriam-webster.com/dictionary/many
The meaning of MANY is consisting of or amounting to a large but indefinite number. How to use many in a sentence.

2. 347 Synonyms & Antonyms for MANY | Thesaurus .com
https://www.thesaurus.com/browse/many
Find 347 different ways to say MANY, along with antonyms, related words, and example sentences at Thesaurus.com.

3. MANY | English meaning - Cambridge Dictionary
https://dictionary.cambridge.org/dictionary/english/many
MANY definition: 1. used mainly in negative sentences and questions and with "too", "so", and "as" to mean "a large…. Learn more.

4. MANY Definition & Meaning | Dictionary .com
https://www.dictionary.com/browse/many
Many definition: constituting or forming a large number; numerous.. See examples of MANY used in a sentence.

5. Many - definition of many by The Free Dictionary
https://www.thefreedictionary.com/many
Define many. many synonyms, many pronunciation, many transla

In [ ]:
results = web_search(query)
for idx, res in enumerate(results, 1):
    print(f"{idx}. {res.title}\n{res.url}\n{res.snippet}\n")

/var/folders/7b/7_944yjs1nbdx3s0b1jsvzyw0000gn/T/ipykernel_41573/2203805329.py:3: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


1. How I Met Your Mother: Ted & Robin's Relationship Timeline, …
https://screenrant.com/how-i-met-your-mother-ted-robins-relationship-timeline-season-by-season/


2. How I Met Your Mother ending explained: Ted's first meeting with …
https://www.soapcentral.com/shows/how-i-met-your-mother-ending-explained-ted-s-first-meeting-mother-barney-robin-s-wedding-flash-forward-twist
1 day ago · Without wasting any time, let's get on to the juiciest part of the How I Met Your Mother finale, the reveal of the Mother and how Ted meets her. While the mother's partial identity is …

3. Ted and Tracy | How I Met Your Mother Wiki | Fandom
https://how-i-met-your-mother.fandom.com/wiki/Ted_and_Tracy
Ted and Tracy officially met in Last Forever - Part Two, this is some information about their relationship. Ted met Tracy at the Farhampton Train Station after Barney and Robin's …

4. The Moment Arrived: When Does Ted Meet His Wife in How I …
https://darkskiesfilm.com/what-episode-does-ted-actually-meet-his-

The agent does decent with generic/fact based queries like the first one, but in case of complex and trivia/storyline based queries like the second, the best it can do is just get a set of the most relevant links. The snippets that it displays are simply not enough to get information fast. This is why we need a RAG acting on the agent's results, and this is focused in other notebooks.